<a href="https://colab.research.google.com/github/KruthikaJ1109/Developing-A-SEPSIS-PREDICTION-App-with-the-help-of-machine-learning-and-FastAPI-Project./blob/main/Bio_mistral_og.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download("stopwords")

import pathlib
import textwrap
from IPython.display import display, Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass("Enter your HuggingFace API Token: ")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


Enter your HuggingFace API Token: ··········


In [7]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("/content/sepsis_book.pdf")
docs = loader.load()

print(docs)


[]


In [26]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the document
loader = PyPDFDirectoryLoader("/content/sepsis_book.pdf")
docs = loader.load()

# Check if documents are loaded
if len(docs) == 0:
    print("No documents loaded. Please check the file path and ensure the PDF is accessible.")
else:
    print(f"Number of documents loaded: {len(docs)}")

# Proceed with text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

print(f"Number of chunks created: {len(chunks)}")
if len(chunks) > 0:
    for i in range(min(1000000, len(chunks))):  # Display up to the first 5 chunks
        print(chunks[i])
else:
    print("No chunks created. Please check the text splitting parameters.")


No documents loaded. Please check the file path and ensure the PDF is accessible.
Number of chunks created: 0
No chunks created. Please check the text splitting parameters.


In [27]:
# Import necessary libraries
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the document
loader = PyPDFLoader("/content/sepsis_book.pdf")
docs = loader.load()

# Verify the document content
print(docs)

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

# Display the number of chunks and the first few chunks
print(len(chunks))
for i in range(min(5, len(chunks))):  # Ensure not to exceed the number of chunks
    print(chunks[i])


Exception ignored in: <function Llama.__del__ at 0x78ea8fb360e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py", line 2089, in __del__
    if self._lora_adapter is not None:
AttributeError: 'Llama' object has no attribute '_lora_adapter'
Exception ignored in: <function Llama.__del__ at 0x78ea8fb360e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py", line 2089, in __del__
    if self._lora_adapter is not None:
AttributeError: 'Llama' object has no attribute '_lora_adapter'
Exception ignored in: <function Llama.__del__ at 0x78ea8fb360e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py", line 2089, in __del__
    if self._lora_adapter is not None:
AttributeError: 'Llama' object has no attribute '_lora_adapter'
Exception ignored in: <function Llama.__del__ at 0x78ea8fb360e0>
Traceback (most recent call last):
  File "/usr/local

[Document(metadata={'source': '/content/sepsis_book.pdf', 'page': 0}, page_content='Andrew C. Liu is a medical student at Donald and Barbara School of Medicine at Hofstra/Northwell, Northwell Health and worked as a health informatics\nintern at Department of Information Services, Northwell Health, New Hyde Park.Krishna Patel is a medical student at Donald and Barbara School of Medicine at Hofstra/Northwell, Northwell Health and worked as a health informatics\nintern at Department of Information Services, Northwell Health, New Hyde Park.\nRamya Dhatri Vunikili is a Master’s student at Courant Institute of Mathematical Sciences, New York University, New York and a summer intern at Center\nfor Research Informatics and Innovation, Northwell Health, New Hyde Park.\nKipp W. Johnson is an MD–PhD student in the Dudley Laboratory at the Icahn School of Medicine at Mount Sinai, Mount Sinai Health System, New\nYork, NY.\nFahad Abdu is a pre-med student and undergraduate at Stonybrook University a

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

print(len(chunks))
for i in range(5):  # Display the first 5 chunks
    print(chunks[i])


311
page_content='Andrew C. Liu is a medical student at Donald and Barbara School of Medicine at Hofstra/Northwell, Northwell Health and worked as a health informatics' metadata={'source': '/content/sepsis_book.pdf', 'page': 0}
page_content='intern at Department of Information Services, Northwell Health, New Hyde Park.Krishna Patel is a medical student at Donald and Barbara School of Medicine at Hofstra/Northwell, Northwell Health and worked as a health informatics' metadata={'source': '/content/sepsis_book.pdf', 'page': 0}
page_content='intern at Department of Information Services, Northwell Health, New Hyde Park.
Ramya Dhatri Vunikili is a Master’s student at Courant Institute of Mathematical Sciences, New York University, New York and a summer intern at Center' metadata={'source': '/content/sepsis_book.pdf', 'page': 0}
page_content='for Research Informatics and Innovation, Northwell Health, New Hyde Park.
Kipp W. Johnson is an MD–PhD student in the Dudley Laboratory at the Icahn Sch

In [15]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
vectorstore = Chroma.from_documents(chunks, embeddings)

query = "who is at risk of sepsis?"
search = vectorstore.similarity_search(query)

to_markdown(search[0].page_content)


> triggering sepsis. Septicemia, a bloodstream infection, which
> involves dangerous bacteria and toxins transporting throughthe human body, that can eventually lead to sepsis if left
> untreated since after all sepsis is a severe complication of

In [18]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

retriever.get_relevant_documents(query)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'page': 9, 'source': '/content/sepsis_book.pdf'}, page_content='triggering sepsis. Septicemia, a bloodstream infection, which\ninvolves dangerous bacteria and toxins transporting throughthe human body, that can eventually lead to sepsis if left\nuntreated since after all sepsis is a severe complication of'),
 Document(metadata={'page': 9, 'source': '/content/sepsis_book.pdf'}, page_content='triggering sepsis. Septicemia, a bloodstream infection, which\ninvolves dangerous bacteria and toxins transporting throughthe human body, that can eventually lead to sepsis if left\nuntreated since after all sepsis is a severe complication of'),
 Document(metadata={'page': 9, 'source': '/content/sepsis_book.pdf'}, page_content='than 30 million people worldwide every year with highmortality rates (15–20%).\n•Irrespective of advances in translational bioinformatics,\nsepsis remains to be an understudied theme comparedto cancer and cardiovascular diseases.\n•Understanding the clinic

In [30]:
from google.colab import drive

drive.mount('/content/drive')

llm = LlamaCpp(
    model_path= "/content/drive/MyDrive/ggml-model-Q4_K_M.gguf",
    temperature=0.3,
    max_tokens=2048,
    top_p=1
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/drive/MyDrive/ggml-model-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """
You are an AI assistant that follows instructions extremely well.
Please be truthful and give direct answers
</s>

{query}
</s>
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("what are the symptoms of sepsis?")

to_markdown(response)


In [ ]:
import sys

while True:
    user_input = input("Input Prompt: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        sys.exit()
    if user_input == '':
        continue
    result = rag_chain.invoke(user_input)
    print("Answer: ", result)


Input Prompt: what are the symptoms of sepsis


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4645.53 ms
llama_print_timings:      sample time =      74.76 ms /   107 runs   (    0.70 ms per token,  1431.32 tokens per second)
llama_print_timings: prompt eval time =    7889.40 ms /    11 tokens (  717.22 ms per token,     1.39 tokens per second)
llama_print_timings:        eval time =   84808.47 ms /   106 runs   (  800.08 ms per token,     1.25 tokens per second)
llama_print_timings:       total time =   92893.84 ms /   117 tokens


Answer:  
The symptoms of sepsis can vary depending on the severity of the infection, but they typically include fever (above 101°F or 38°C), chills, rapid heart rate and breathing, confusion, and low blood pressure. In more severe cases, there may be organ failure, such as kidney or liver failure, and septic shock, which is a life-threatening condition. It's important to seek medical attention if you suspect you have sepsis.
Input Prompt:  Hi


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4645.53 ms
llama_print_timings:      sample time =       5.88 ms /    10 runs   (    0.59 ms per token,  1700.39 tokens per second)
llama_print_timings: prompt eval time =    2615.00 ms /     5 tokens (  523.00 ms per token,     1.91 tokens per second)
llama_print_timings:        eval time =    6367.46 ms /     9 runs   (  707.50 ms per token,     1.41 tokens per second)
llama_print_timings:       total time =    8998.03 ms /    14 tokens


Answer:  Hello! How can I assist you today?
Input Prompt: HI


...

Answer:  
Sepsis is a life-threatening condition that occurs when the body's response to infection causes damage to its own tissues and organs. There is no specific cure for sepsis, but early diagnosis and treatment are critical in improving outcomes. Treatment typically involves antibiotics to kill the bacteria causing the infection, as well as supportive care such as fluid replacement and oxygen therapy to help maintain blood pressure and breathing. In severe cases, patients may require additional treatments such as vasopressors or mechanical ventilation.
Input Prompt: Is lactate levels very important for predicting sepsis


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4645.53 ms
llama_print_timings:      sample time =      84.28 ms /   119 runs   (    0.71 ms per token,  1411.88 tokens per second)
llama_print_timings: prompt eval time =   10087.56 ms /    16 tokens (  630.47 ms per token,     1.59 tokens per second)
llama_print_timings:        eval time =   92693.04 ms /   119 runs   (  778.93 ms per token,     1.28 tokens per second)
llama_print_timings:       total time =  102995.64 ms /   135 tokens


Answer:  
Lactate is a biomarker of tissue hypoxia, which can occur in various conditions including sepsis. Elevated lactate levels are associated with increased mortality and morbidity in patients with sepsis. However, it is important to note that lactate levels can be affected by many factors other than sepsis, such as hypovolemia, liver dysfunction, and medications. Therefore, while lactate levels may be useful in predicting sepsis, they should not be used in isolation for this purpose.
Input Prompt: Thank you, bye


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4645.53 ms
llama_print_timings:      sample time =       3.97 ms /     5 runs   (    0.79 ms per token,  1259.45 tokens per second)
llama_print_timings: prompt eval time =    3430.10 ms /     8 tokens (  428.76 ms per token,     2.33 tokens per second)
llama_print_timings:        eval time =    3605.79 ms /     5 runs   (  721.16 ms per token,     1.39 tokens per second)
llama_print_timings:       total time =    7045.58 ms /    13 tokens


Answer:  
Goodbye!


In [ ]:
import sys

while True:
    user_input = input("Input Prompt: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        sys.exit()
    if user_input == '':
        continue
    result = rag_chain.invoke(user_input)
    print("Answer: ", result)
